# TP : Conception d'Algorithmes Génétiques avec GeneticSharp

## Objectif
Dans ce TP, nous allons concevoir un filtre de détection de bords en utilisant la bibliothèque **GeneticSharp**. Nous chercherons à approximer un filtre de convolution de référence en évoluant une population de filtres grâce à un algorithme génétique.

---


In [1]:
// Dépendances nécessaires
#r "nuget: GeneticSharp, 3.1.4"
#r "nuget: System.Drawing.Common, 9.0.0"

#r "nuget: Emgu.CV, 4.9.0.5494"
#r "nuget: Emgu.CV.Bitmap, 4.9.0.5494"
#r "nuget: Emgu.CV.runtime.mini.windows, 4.9.0.5494"

#r "nuget: SkiaSharp, 2.88.3"

#load "../Config/SkiaUtils.cs"


In [ ]:
// Imports
using System;
using System.IO;
using System.Drawing;
using Microsoft.DotNet.Interactive;

using System.Diagnostics;
using System.Globalization;

using GeneticSharp;

using Emgu.CV;
using Emgu.CV.CvEnum;
using Emgu.CV.Structure;

public static void SaveImage(Mat image, string fileName)
{
    image.Save(fileName);
    Console.WriteLine($"Image enregistrée : {fileName}");
}

Console.WriteLine("Dépendances chargées.");

## Étape 1 : Définir le Chromosome

Nous allons implémenter un chromosome pour représenter les filtres par convolution. Chaque chromosome contiendra des gènes, qui sont des matrices symétriques. Ces matrices seront sommées pour produire le filtre final.

---


In [2]:
// Représentation du filtre par convolution sous forme de chromosome
public class EdgeChromosome : ChromosomeBase
{
    private const int KernelSize = 7; // Taille de la matrice du noyau

    public EdgeChromosome(int length) : base(length)
    {
        // Initialisation des gènes
        for (int i = 0; i < Length; i++)
        {
            ReplaceGene(i, GenerateGene(i));
        }
    }

    // Création d'un nouveau chromosome pour la population
    public override IChromosome CreateNew()
    {
        return new EdgeChromosome(Length);
    }

    // Génération d'un gène contenant une matrice avec des perturbations asymétriques
    public override Gene GenerateGene(int geneIndex)
    {
        var rnd = RandomizationProvider.Current;
        var matrix = new int[KernelSize, KernelSize];

        for (int i = 0; i < KernelSize; i++)
        {
            for (int j = 0; j < KernelSize; j++)
            {
                matrix[i, j] = rnd.GetInt(-20, 20);
               
            }
        }
        return new Gene(matrix);
    }

    public int[,] GetCompleteMatrix()
{
    var completeMatrix = new int[KernelSize, KernelSize];

    // Ajouter les matrices des gènes
    foreach (var gene in GetGenes())
    {
        var matrix = (int[,])gene.Value;
        for (int i = 0; i < KernelSize; i++)
        {
            for (int j = 0; j < KernelSize; j++)
            {
                completeMatrix[i, j] += matrix[i, j];
            }
        }
    }

    // Normalisation dynamique si des valeurs extrêmes apparaissent
    int maxAbsValue = completeMatrix.Cast<int>().Select(Math.Abs).Max();
    if (maxAbsValue > 10) // Seulement si les valeurs dépassent un seuil
    {
        for (int i = 0; i < KernelSize; i++)
        {
            for (int j = 0; j < KernelSize; j++)
            {
                completeMatrix[i, j] = (int)(10.0 * completeMatrix[i, j] / maxAbsValue);
            }
        }
    }

    return completeMatrix;
}

}
Console.WriteLine("Chromosome défini.");


Testons la création d'un chromosome, ses gènes, et la matrice résultante correspondante

In [ ]:
var testChromosome = new EdgeChromosome(5); // 5 gènes
Console.WriteLine("Gènes générés :");
foreach (var gene in testChromosome.GetGenes())
{
    var matrix = (int[,])gene.Value;
    for (int i = 0; i < matrix.GetLength(0); i++)
    {
        for (int j = 0; j < matrix.GetLength(1); j++)
        {
            Console.Write($"{matrix[i, j]} ");
        }
        Console.WriteLine();
    }
    Console.WriteLine();
}

var testMatrix = testChromosome.GetCompleteMatrix();

Console.WriteLine("Matrice complète générée :");
for (int i = 0; i < testMatrix.GetLength(0); i++)
{
    for (int j = 0; j < testMatrix.GetLength(1); j++)
    {
        Console.Write($"{testMatrix[i, j]} ");
    }
    Console.WriteLine();
}


## Étape 2 : Définir la Fonction d'Évaluation

Nous allons implémenter une classe qui compare les résultats des filtres générés par les chromosomes à un filtre de référence basé sur Sobel.

---


In [3]:
using Emgu.CV;
using Emgu.CV.CvEnum;
using Emgu.CV.Structure;
using System.Drawing;
using System.Runtime.InteropServices;

public class EdgeFitness : IFitness
{
    private readonly Mat _originalImage;
    private readonly Mat _referenceImage;

    public EdgeFitness(Bitmap originalImage)
    {
    _originalImage = BitmapToMat(originalImage);

    // Convertir en niveaux de gris
    if (_originalImage.NumberOfChannels > 1)
    {
        CvInvoke.CvtColor(_originalImage, _originalImage, ColorConversion.Bgr2Gray);
    }

    // Appliquer le filtre Sobel pour référence
    _referenceImage = new Mat();
    CvInvoke.Sobel(_originalImage, _referenceImage, DepthType.Cv64F, 1, 0);

    if (_referenceImage.NumberOfChannels != 1)
        {
            CvInvoke.CvtColor(_referenceImage, _referenceImage, ColorConversion.Bgr2Gray);
        }

    }


    public async Task DisplayImagesAsync()
    {
        // Afficher l'image originale
        SaveImage(_originalImage, "original.png");
        await SkiaUtils.ShowImage("original.png", _originalImage.Width, _originalImage.Height);

        // Afficher l'image de référence (filtre Sobel)
        SaveImage(_referenceImage, "reference.png");
        await SkiaUtils.ShowImage("reference.png", _referenceImage.Width, _referenceImage.Height);
    }


    

    private Mat ApplyFilter(EdgeChromosome chromosome, bool display = false)
    {
        var filterMatrix = chromosome.GetCompleteMatrix();
        var kernel = ArrayToMat(ConvertToFloat(filterMatrix));

        if (display)
        {   
            Console.WriteLine("Noyau généré :");
            for (int i = 0; i < kernel.Rows; i++)
            {
                for (int j = 0; j < kernel.Cols; j++)
                {
                    Console.Write($"{kernel.GetData().GetValue(i, j)} ");
                }
                Console.WriteLine();
            }
        }
        

        var sourceImage = _originalImage.Clone();
        if (sourceImage.Depth != DepthType.Cv32F)
        {
            sourceImage.ConvertTo(sourceImage, DepthType.Cv32F);
        }

        var filteredImage = new Mat(sourceImage.Rows, sourceImage.Cols, DepthType.Cv32F, 1);
        CvInvoke.Filter2D(sourceImage, filteredImage, kernel, new Point(-1, -1));

        // Aligner la profondeur avec _referenceImage
        if (filteredImage.Depth != _referenceImage.Depth)
        {
            filteredImage.ConvertTo(filteredImage, _referenceImage.Depth);
        }

        return filteredImage;
    }


    
    public double Evaluate(IChromosome chromosome)
{
    var filteredImage = ApplyFilter((EdgeChromosome)chromosome, false);

    // Convertir en niveaux de gris si nécessaire
    if (filteredImage.NumberOfChannels > 1)
    {
        CvInvoke.CvtColor(filteredImage, filteredImage, ColorConversion.Bgr2Gray);
    }

    if (_referenceImage.NumberOfChannels > 1)
    {
        CvInvoke.CvtColor(_referenceImage, _referenceImage, ColorConversion.Bgr2Gray);
    }

    // Assurer la même profondeur
    if (filteredImage.Depth != DepthType.Cv8U)
    {
        filteredImage.ConvertTo(filteredImage, DepthType.Cv8U);
    }

    if (_referenceImage.Depth != DepthType.Cv8U)
    {
        _referenceImage.ConvertTo(_referenceImage, DepthType.Cv8U);
    }

    // Redimensionner si nécessaire
    if (filteredImage.Size != _referenceImage.Size)
    {
        CvInvoke.Resize(filteredImage, filteredImage, _referenceImage.Size);
    }

    // Créer une matrice pour stocker le résultat
    var result = new Mat();

    // Appliquer la méthode de corrélation normalisée
    CvInvoke.MatchTemplate(filteredImage, _referenceImage, result, TemplateMatchingType.CcorrNormed);

    // Extraire le score de corrélation maximum
    double minVal = 0, maxVal = 0;
    Point minLoc = new Point(), maxLoc = new Point();
    CvInvoke.MinMaxLoc(result, ref minVal, ref maxVal, ref minLoc, ref maxLoc);

    // Pénalisation des filtres uniformes
    var filterMatrix = ((EdgeChromosome)chromosome).GetCompleteMatrix();
    var filterSum = filterMatrix.Cast<int>().Sum();
    var filterPenalty = Math.Abs(filterSum) < 1e-3 ? 1 : Math.Log10(Math.Abs(filterSum) + 1);

    // Retourner le score ajusté
    return maxVal / filterPenalty;
}



    public async Task<DisplayedValue> DisplayChromosomeResult(EdgeChromosome chromosome, string fileNamePrefix, int generation, DisplayedValue placeholder = null)
{
    try
    {
        var filteredImage = ApplyFilter(chromosome, true);

        // Sauvegarder l'image
        string fileName = $"{fileNamePrefix}_generation_{generation}.png";
        SaveImage(filteredImage, fileName);
        Console.WriteLine($"Image sauvegardée : {fileName}");

        if (placeholder != null)
        {
            placeholder.Update(await SkiaUtils.ShowImage(fileName, filteredImage.Width, filteredImage.Height));
            return placeholder;
        }

        return await SkiaUtils.ShowImage(fileName, filteredImage.Width, filteredImage.Height);
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Erreur dans DisplayChromosomeResult : {ex.Message}");
        throw;
    }
}


    private Mat BitmapToMat(Bitmap bitmap)
    {
        // Crée un Mat vide avec les mêmes dimensions et type
        var mat = new Mat(bitmap.Height, bitmap.Width, DepthType.Cv8U, 3);

        // Bloquer les bits du Bitmap pour accéder directement aux données
        var bitmapData = bitmap.LockBits(
            new Rectangle(0, 0, bitmap.Width, bitmap.Height),
            System.Drawing.Imaging.ImageLockMode.ReadOnly,
            System.Drawing.Imaging.PixelFormat.Format24bppRgb);

        // Copier les données du Bitmap vers le Mat
        using (var image = new Image<Bgr, byte>(bitmap.Width, bitmap.Height, bitmapData.Stride, bitmapData.Scan0))
        {
            mat = image.Mat.Clone();
        }

        // Libérer les bits verrouillés
        bitmap.UnlockBits(bitmapData);

        return mat;
    }
    

   private Mat ArrayToMat(float[,] array)
    {
        var rows = array.GetLength(0);
        var cols = array.GetLength(1);
        var mat = new Mat(rows, cols, DepthType.Cv32F, 1);

        var data = new float[rows * cols];
        Buffer.BlockCopy(array, 0, data, 0, rows * cols * sizeof(float));
        mat.SetTo(data);

        return mat;
    }


    private float[,] ConvertToFloat(int[,] intArray)
    {
        var rows = intArray.GetLength(0);
        var cols = intArray.GetLength(1);
        var floatArray = new float[rows, cols];

        for (int i = 0; i < rows; i++)
        {
            for (int j = 0; j < cols; j++)
            {
                floatArray[i, j] = intArray[i, j];
            }
        }
        return floatArray;
    }
}


Test de la fonction fitness

In [ ]:
var imagePath = @"E:\Dev\AI\Cours\CoursIA\MyIA.AI.Notebooks\Search\MRI_Prostate_Cancer.jpg"; 
var originalImage = (Bitmap)Image.FromFile(imagePath);

var fitness = new EdgeFitness(originalImage);

// Test d'un chromosome
var chromosome = new EdgeChromosome(20);
Console.WriteLine($"Score de fitness : {fitness.Evaluate(chromosome)}");


## Étape 3 : Configuration de l'Algorithme Génétique

Nous configurons l'algorithme génétique avec une population, une fonction de sélection, un opérateur de croisement et un opérateur de mutation.

---


In [4]:
// Charger une image de test
var imagePath = @"E:\Dev\AI\Cours\CoursIA\MyIA.AI.Notebooks\Search\MRI_Prostate_Cancer.jpg"; 
var originalImage = (Bitmap)Bitmap.FromFile(imagePath);

// await SkiaUtils.ShowImage(imagePath, originalImage.Width, originalImage.Height);

// Initialiser la fonction de fitness
var fitness = new EdgeFitness(originalImage);
await fitness.DisplayImagesAsync();

// Initialiser un chromosome
var chromosome = new EdgeChromosome(20); // Taille des chromosomes

// Initialiser la population
var population = new Population(50, 100, chromosome);

// Configurer les opérateurs génétiques
var selection = new EliteSelection();
var crossover = new UniformCrossover();
var mutation = new ReverseSequenceMutation();

// Configurer l'algorithme génétique
var ga = new GeneticAlgorithm(population, fitness, selection, crossover, mutation)
{
    Termination = new GenerationNumberTermination(100)

};


Console.WriteLine("Algorithme génétique configuré.");


## Étape 4 : Exécution et Visualisation

Nous lançons l'algorithme génétique et suivons l'évolution des solutions.

---


In [5]:
// Initialiser le placeholder avec l'image originale
DisplayedValue imagePlaceholder = await SkiaUtils.ShowImage(imagePath, originalImage.Width, originalImage.Height);

ga.GenerationRan += (sender, e) =>
{
    var bestChromosome = ga.BestChromosome as EdgeChromosome;
    if (bestChromosome != null)
    {
        Console.WriteLine($"Génération {ga.GenerationsNumber} - Meilleur score : {bestChromosome.Fitness}");

        if (ga.GenerationsNumber % 10 == 0)
        {
            // Mettre à jour l'image via DisplayChromosomeResult
            Task.Run(async () =>
            {
                try
                {
                    imagePlaceholder = await fitness.DisplayChromosomeResult(bestChromosome, "best", ga.GenerationsNumber).ConfigureAwait(false);
                }
                catch (Exception ex)
                {
                    Console.WriteLine($"Erreur lors de la mise à jour de l'image : {ex.Message}");
                }
            });
        }
    }
};



Console.WriteLine("Lancement de l'algorithme génétique...");
ga.Start();
Console.WriteLine($"Meilleure solution trouvée avec un score de {ga.BestChromosome.Fitness}.");


## Conclusion

Ce TP vous a permis de découvrir les algorithmes génétiques appliqués à un problème de traitement d'images. Vous pouvez prolonger cette activité en modifiant les paramètres du GA ou en testant d'autres types de filtres de convolution.

---

**Prochaine étape :** Implémenter une version équivalente en Python avec PyGad !
